In [11]:
import os
import tqdm
import pandas as pd

In [3]:
docFolderDir = './S04_AnswerScoring/'
lsDocDir = [f for f in os.listdir(docFolderDir)]
lsDocDir

['Industrial_Gas_Turbines_AMY_Razak',
 'msd_servo_drive',
 'pcs7_compendium_part_a_en-US_en-US',
 'pcs7_compendium_part_b_en-US_en-US']

In [36]:
lsALL = []

for i in range(len(lsDocDir)):
    docfolder = lsDocDir[i]
    print(docfolder)
    # LIST ALL CSV IN FOLDER
    lscsv = [f for f in os.listdir(f"./S04_AnswerScoring/{docfolder}") if f.endswith('.csv')]
    # LOAD INTO BIG DF
    lsdf = []
    for j in tqdm.tqdm(range(len(lscsv))):
        csvname = lscsv[j]
        df = pd.read_csv(f"./S04_AnswerScoring/{docfolder}/{csvname}")
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0', axis=1)
        lsdf.append(df)
    # CONCAT
    dfANS = pd.concat(lsdf)
    # WEIGHTED SCORE
    dfANS['WEIGHTED_SCORE'] = (0.3*dfANS['helpfulness']) + (0.3*dfANS['correctness']) + (0.2*dfANS['coherence']) + (0.1*dfANS['complexity']) + (-0.1*dfANS['verbosity'])
    # MARK BEST SCORE
    dfANS['MAX'] = dfANS.groupby(['NODE_HASH', 'Q'])['WEIGHTED_SCORE'].transform(lambda x: x == x.max()).replace({True: 'Y', False: ''})
    # FILTER ONLY BEST
    dfANS = dfANS[dfANS['MAX'] == 'Y']
    # SAVE
    dfANS.to_csv(f"./S05_SelectGoodScore/{docfolder}.csv", index=False)
    lsALL.append(dfANS)

# CONCAT ALL
dfALLALL = pd.concat(lsALL)
dfALLALL = dfALLALL[['NODE_FILE_NAME','NODE_NUMBER','NODE_HASH','Q','A','WEIGHTED_SCORE']]
# SAVE
dfALLALL.to_csv(f"./S05_SelectGoodScore/ALLALL.csv", index=False)

Industrial_Gas_Turbines_AMY_Razak


100%|██████████| 381/381 [00:01<00:00, 253.33it/s]


msd_servo_drive


100%|██████████| 250/250 [00:00<00:00, 311.35it/s]


pcs7_compendium_part_a_en-US_en-US


100%|██████████| 116/116 [00:00<00:00, 294.11it/s]


pcs7_compendium_part_b_en-US_en-US


100%|██████████| 81/81 [00:00<00:00, 259.83it/s]


In [35]:
dfALL

,NODE_FILE_NAME,NODE_NUMBER,NODE_HASH,NODE_TEXT,Q,A_NUMBER,A,helpfulness,correctness,coherence,complexity,verbosity,WEIGHTED_SCORE,MAX
3,Industrial_Gas_Turbines_AMY_Razak.txt,0,db0a427dd2a71cf3886fd3945a64acc8ea61ce17ca75bc...,# Industrial gas turbines\r\n\r\n# Performance...,What are the main components of an industrial ...,A4,"Heat exchanger, combustor, compressor, gas sup...",1.773438,1.835938,3.531250,1.398438,0.429688,1.885938,Y
7,Industrial_Gas_Turbines_AMY_Razak.txt,0,db0a427dd2a71cf3886fd3945a64acc8ea61ce17ca75bc...,# Industrial gas turbines\r\n\r\n# Performance...,What does the CD-ROM included with the book 'I...,A3,Materials provided by Gas Path Analysis Ltd ar...,1.140625,1.117188,2.984375,1.039062,0.357422,1.342383,Y
14,Industrial_Gas_Turbines_AMY_Razak.txt,0,db0a427dd2a71cf3886fd3945a64acc8ea61ce17ca75bc...,# Industrial gas turbines\r\n\r\n# Performance...,What is the web address of Woodhead Publishing...,A5,The book lists www.woodheadpublishing.com as t...,2.859375,2.968750,4.031250,0.671875,0.412109,2.580664,Y
19,Industrial_Gas_Turbines_AMY_Razak.txt,0,db0a427dd2a71cf3886fd3945a64acc8ea61ce17ca75bc...,# Industrial gas turbines\r\n\r\n# Performance...,What locations are mentioned as publishing sit...,A5,"Abington Hall, Cambridge, England, is where Wo...",1.367188,1.484375,3.328125,0.839844,0.589844,1.546094,Y
21,Industrial_Gas_Turbines_AMY_Razak.txt,0,db0a427dd2a71cf3886fd3945a64acc8ea61ce17ca75bc...,# Industrial gas turbines\r\n\r\n# Performance...,What rights does the author assert in the book...,A2,A. M. Y Razak asserts his moral rights in the ...,1.953125,2.171875,3.640625,0.867188,0.410156,2.011328,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,pcs7_compendium_part_b_en-US_en-US.txt,9,d106a69be20bf6bad15c6871f028013d3eafb348c28a93...,2. This is followed by logon to the SIMATIC Lo...,What indicates that a project within a multipr...,A1,The project is displayed grayed-out when the m...,3.093750,3.187500,3.859375,1.328125,0.574219,2.731641,Y
30,pcs7_compendium_part_b_en-US_en-US.txt,9,d106a69be20bf6bad15c6871f028013d3eafb348c28a93...,2. This is followed by logon to the SIMATIC Lo...,What is the first step in accessing the SIMATI...,A1,The first step is logging on to the SIMATIC Lo...,2.421875,2.390625,3.546875,1.093750,0.507812,2.211719,Y
39,pcs7_compendium_part_b_en-US_en-US.txt,9,d106a69be20bf6bad15c6871f028013d3eafb348c28a93...,2. This is followed by logon to the SIMATIC Lo...,What version of PCS 7 corresponds to STEP 7 as...,A5,"For STEP 7 V5.5.4 Upd9, the matching PCS 7 ver...",3.015625,3.390625,3.859375,1.273438,0.472656,2.773828,Y
43,pcs7_compendium_part_b_en-US_en-US.txt,9,d106a69be20bf6bad15c6871f028013d3eafb348c28a93...,2. This is followed by logon to the SIMATIC Lo...,When is the project password set in the SIMATI...,A4,The password for the project is configured whe...,2.390625,2.609375,3.703125,1.164062,0.396484,2.317383,Y
